In [0]:
# Commands to Install Required libraries
pip install docx2txt
pip install PyMuPDF

  Created wheel for docx2txt: filename=docx2txt-0.8-cp36-none-any.whl size=3963 sha256=d64ec0ec8d101127af50208c98b9478c84e2c453696320ff4cb30a3b034f51aa
  Stored in directory: /root/.cache/pip/wheels/b2/1f/26/a051209bbb77fc6bcfae2bb7e01fa0ff941b82292ab084d596
Successfully built docx2txt


In [0]:
#importing the libraries

import os
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

import docx2txt
import en_core_web_sm
nlp = en_core_web_sm.load()
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd
import numpy as np
from time import time 
import multiprocessing
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split


import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
import fitz


nltk.download('wordnet')


nltk.download('punkt')

Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
#Tokenization of files in filePath 

def getTokensFromFiles(filePath, label):
  main=[]
  for i in os.listdir(filePath):
    if i != ".ipynb_checkpoints" and i.endswith('pdf') == False:
      x=i
      try:
        if x.endswith('docx'):
          fname = docx2txt.process(filePath + "/" + x)
        elif x.endswith('pdf'):
          fname = ""
          doc = fitz.open(filePath +"/" +x)
          pages = doc.pageCount 
          for i in range(0,pages):
            page1 = doc.loadPage(i)
            fname += page1.getText("text")
        else:  
          fname = open(filePath +"/"+ x).read()
      except UnicodeDecodeError:
        continue
          
      f1=re.sub(r'\W+', ' ',fname)
      
      wordnet_lemmatizer = WordNetLemmatizer()
      
      words = nltk.word_tokenize(f1)
      
      f2=" "
      for w in words:
        f2=f2+" "+wordnet_lemmatizer.lemmatize(w)
    
      doc = nlp(f2)
      
      tokens = [token.text for token in doc if not token.is_stop]
      
      l=[]
      l.append(tokens)
      if label != -1:
        l.append(label)
      main.append(l)

  return main



In [0]:
import tensorflow as tf

# Training the word2vec model with the preprocessed trainset
def trainw2v(trainSet):
  print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
  cores = len(tf.config.experimental.list_physical_devices('GPU'))
  w2v_model = Word2Vec(min_count=20,
                       window=2,
                       size=1000,
                       sample=6e-5, 
                       alpha=0.03, 
                       min_alpha=0.0007, 
                       negative=20,
                       workers=cores-1)

  t = time()
  w2v_model.build_vocab(trainSet["Tokens"], progress_per=1000)
  print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

  w2v_model.train(trainSet["Tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
  print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

  return w2v_model;

In [0]:
#Building the word vector 

def buildWordVector(w2v_model, tfidf, tokens, size):
    vec = np.zeros(size).reshape((1, size))
    
    count = 0.
    for word in tokens:
        try:
    
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    
    
    return vec


In [0]:
# Train and save the DNN model
def saveModel(train_vecs_w2v, y_train, test_vecs_w2v, y_test):
  dnn_model = Sequential()
  dnn_model.add(Dense(128, activation='relu', input_dim=1000))
  dnn_model.add(Dropout(0.7))
  dnn_model.add(Dense(1, activation='sigmoid'))
  dnn_model.compile(optimizer='adadelta',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  dnn_model.summary()
  earlystop = EarlyStopping(monitor = 'loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)  
  history = dnn_model.fit(train_vecs_w2v, y_train, epochs=60, batch_size=50,validation_data=(test_vecs_w2v,y_test), callbacks=[earlystop])
  loss, accuracy = dnn_model.evaluate(train_vecs_w2v, y_train, verbose=False)
  print("Training Accuracy: {:.4f}".format(accuracy))
  loss, accuracy = dnn_model.evaluate(test_vecs_w2v, y_test, verbose=False)
  print("Testing Accuracy:  {:.4f}".format(accuracy))
  dnn_model.save("/content/sample_data/DNN_Model", overwrite=True, include_optimizer=True)



In [13]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#Preparing tokens from raw data

with tf.device('/GPU:0'):
  training_test = []
  training_test = training_test + getTokensFromFiles("/content/drive/My Drive/resume/Training & Test/Resume", 1)
  training_test = training_test + getTokensFromFiles("/content/drive/My Drive/resume/Training & Test/Non Resume", 0)
  prediction = getTokensFromFiles("/content/drive/My Drive/resume/Prediction /Resume", -1)
  prediction = prediction + getTokensFromFiles("/content/drive/My Drive/resume/Prediction /Non Resume", -1)


  #Creating Dataframes for Training & prediction tokens


  train_df = pd.DataFrame(training_test, columns = ['Tokens', 'Label'])
  pred_df = pd.DataFrame(prediction, columns = ['Tokens'])



Num GPUs Available:  1


In [15]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


#Train Word2Vec model with personal data

with tf.device('/GPU:0'):
  trained_w2v_model = trainw2v(train_df)

y = train_df['Label'].values
X = np.array(train_df["Tokens"])

#Split the training and test set in provided ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_pred = np.array(pred_df["Tokens"])

vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)

matrix = vectorizer.fit_transform([x for x in X_train])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))


Num GPUs Available:  1
Num GPUs Available:  1
Time to build vocab: 0.0 mins
Time to train the model: 0.09 mins
vocab size : 920


In [16]:
import numpy as np

#Build the word vectors for train, test & prediction dataset 
train_vecs_w2v = np.concatenate([buildWordVector(trained_w2v_model, tfidf , z, 1000) for z in map(lambda x: x, X_train)])
test_vecs_w2v = np.concatenate([buildWordVector(trained_w2v_model, tfidf , z, 1000) for z in map(lambda x: x, X_test)])
pred_vec_w2v = np.concatenate([buildWordVector(trained_w2v_model, tfidf , z, 1000) for z in map(lambda x: x, X_pred)])

#Scale the vectors
train_vecs_w2v = scale(train_vecs_w2v)
test_vecs_w2v = scale(test_vecs_w2v)
pred_vec_w2v = scale(pred_vec_w2v)


print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape,
       '\nshape for predicted set : ', pred_vec_w2v.shape,) 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


shape for training set :  (208, 1000) 
shape for test set :  (90, 1000) 
shape for predicted set :  (2, 1000)


In [17]:
#Train and save the DNN model 

from keras.callbacks import EarlyStopping 
saveModel(train_vecs_w2v, y_train, test_vecs_w2v, y_test)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               128128    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 128,257
Trainable params: 128,257
Non-trainable params: 0
_________________________________________________________________



Train on 208 samples, validate on 90 samples
Epoch 1/60





208/208 [==============================] - 15s 74ms/step - loss: 0.8948 - acc: 0.6154 - val_loss: 0

In [18]:
from keras.models import load_model

trained_model = load_model("/content/sample_data/DNN_Model")

l = trained_model.predict(pred_vec_w2v, verbose=True)
print(l)

2/2 [==============================] - 0s 25ms/step
[[0.93321955]
 [0.1075175 ]]
